# Initialize

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# find packages'version before Apr 25 (article public)
!pip install langchain==0.0.147 python-dotenv huggingface_hub==0.14.0 sentence_transformers==2.2.2 youtube-transcript-api==0.6.0 pytube==12.1.3 faiss-cpu==1.7.4

- Source: [Getting Started with LangChain: A Beginner’s Guide to Building LLM-Powered Applications](https://towardsdatascience.com/getting-started-with-langchain-a-beginners-guide-to-building-llm-powered-applications-95fc8898732c#b842) - *Leonie Monigatti*

- [ ] Watch next: [ LangChain - Using Hugging Face Models locally (code walkthrough) ](https://www.youtube.com/watch?v=Kn7SX2Mx_Jk)
  - [ ] try Blenderbot

# Logging, test

In [ ]:
envfile = open("/content/drive/My Drive/Colab Files/.env", "r")
print(envfile.read())
envfile.close()

In [ ]:
envfile = open("/content/drive/My Drive/Colab Files/requirements.txt", "r")
print(envfile.read())
envfile.close()

In [ ]:
!pip list

# Execution (HF inference APIs with HFHub)

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('/content/drive/My Drive/Colab Files/.env')

True

In [7]:
# Alternatively, open-source LLM hosted on Hugging Face
# pip install huggingface_hub
from langchain import HuggingFaceHub

llm_model = "google/flan-t5-large"
embed_model = "sentence-transformers/all-MiniLM-L6-v2"

llm = HuggingFaceHub(repo_id=llm_model)

# The LLM takes a prompt as an input and outputs a completion
prompt = "Alice has a parrot. What animal is Alice's pet?"
completion = llm(prompt)

print(completion)

parrot


In [8]:
# Alternatively, open-source text embedding model hosted on Hugging Face
# pip install sentence_transformers
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name=embed_model)

# The embeddings model takes a text as an input and outputs a list of floats
text = "Alice has a parrot. What animal is Alice's pet?"
text_embedding = embeddings.embed_query(text)
print(text_embedding)

[0.07190390676259995, 0.07759877294301987, -0.009208649396896362, 0.0561705157160759, -0.09020029753446579, 0.013322998769581318, 0.098457932472229, -0.09042120724916458, 0.07143571972846985, 0.012598483823239803, 0.031594615429639816, -0.0997156873345375, -0.011282620951533318, 0.00715700164437294, -0.004754439927637577, 0.0469106025993824, 0.011661778204143047, -0.06561293452978134, 0.037585847079753876, -0.0009845815366134048, -0.05514022335410118, 0.010666146874427795, 0.06513642519712448, 0.0041189673356711864, -0.028614958748221397, 0.07131151854991913, -0.02405756525695324, -0.028053177520632744, -0.0359695740044117, -0.05724122002720833, -0.059308696538209915, 0.02516922913491726, -0.003160704392939806, 0.008553335443139076, -0.043387722223997116, 0.01663794368505478, -0.007296689786016941, 0.027561135590076447, 0.14499451220035553, 0.04577790945768356, -0.003844153368845582, -0.06782859563827515, 0.008425702340900898, -0.08002766221761703, -0.06505830585956573, 0.0201843958348

In [9]:
from langchain import PromptTemplate

template = "What is a good name for a company that makes {product}?"

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

test_1 = prompt.format(product="colorful socks")
print(test_1)

What is a good name for a company that makes colorful socks?


In [10]:
# use our prompt template in LLMs with `LLMChain`
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, 
                  prompt = prompt)

# Run the chain only specifying the input variable.
test_2 = chain.run("colorful socks")
print(test_2)

sock mania


In [11]:
# chain the first LLM's output to second LLM's input
from langchain.chains import SimpleSequentialChain

# Create a second chain with a prompt template and an LLM
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Tell me a joke for the following company name: {company_name}",
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Combine the first and the second chain 
overall_chain = SimpleSequentialChain(chains=[chain,chain_two]
                                      , verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run('colorful socks')



> Entering new SimpleSequentialChain chain...
sock mania
sock mania is a company that makes socks

> Finished chain.


# Access external data (Q&A task)



In [12]:
# pip install youtube-transcript-api
# pip install pytube

from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=hLQl3WQQoQ0")

documents = loader.load()
print(documents)

[Document(page_content="(relaxing music) (relaxing music continues) ♪ I heard that you're settled down ♪ ♪ That you found a girl ♪ ♪ And you're married now ♪ ♪ I heard that your dreams came true ♪ ♪ Guess she gave you things ♪ ♪ I didn't give to you ♪ ♪ Old friend, why are you so shy ♪ ♪ Ain't like you to hold back ♪ ♪ Or hide from the light ♪ ♪ I hate to turn up out\nof the blue, uninvited ♪ ♪ But I couldn't stay away ♪ ♪ I couldn't fight it ♪ ♪ I had hoped you'd see my face ♪ ♪ And that you'd be reminded ♪ ♪ That for me it isn't over ♪ ♪ Never mind, I'll find someone like you ♪ ♪ I wish nothing but\nthe best for you, too ♪ ♪ Don't forget me I beg ♪ ♪ I remember you said ♪ ♪ Sometimes it lasts in love ♪ ♪ But sometimes it hurts instead ♪ ♪ Sometimes it lasts in love ♪ ♪ But sometimes it hurts instead ♪ ♪ You know how the time flies ♪ ♪ Only yesterday was\nthe time of our lives ♪ ♪ We were born and\nraised in a summer haze ♪ ♪ Bound by the surprise\nof our glory days ♪ ♪ I hate to turn

In [13]:
# index with embedding model + store in vector database (for similar search,...)
# FAISS doesn't require API keys

# pip install faiss-cpu
from langchain.vectorstores import FAISS

# create the vectorestore to use as the index
db = FAISS.from_documents(documents, embeddings)

In [14]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

query = "What's my plan?"
result = qa({"query":query})

print(result['result'])

Never mind, I'll find someone like you
